In [1]:
import cv2 as cv
from glob import glob
import os
import random
from ultralytics import YOLO

In [2]:
coco_model = YOLO('yolov8n.pt')
np_model = YOLO('runs/detect/train5/weights/best.pt')
videos = glob("LicenseVideos/*mp4")

In [3]:
print(videos)

['LicenseVideos/UconnLicense.mp4', 'LicenseVideos/UconnLicense2.mp4']


# Implementing the Car Detection

In [4]:
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
# all vehicle class IDs from the COCO dataset (car, motorbike, truck) https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
vehicles = [2,3,5,6]
vehicle_bounding_boxes = []

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        # use track() to identify instances and track them frame by frame
        detections = coco_model.track(frame, persist=True)[0]
        # save cropped detections
        # detections.save_crop('outputs')
        # print nodel predictions for debugging
        # print(results)

        for detection in detections.boxes.data.tolist():
            # print detection bounding boxes for debugging
            # print(detection)
            x1, y1, x2, y2, track_id, score, class_id = detection
            # I am only interested in class IDs that belong to vehicles
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])

# print found bounding boxes for debugging
print(vehicle_bounding_boxes)
video.release()


0: 384x640 2 cars, 77.0ms
Speed: 6.2ms preprocess, 77.0ms inference, 80.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
Speed: 2.1ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.9ms
Speed: 2.0ms preprocess, 5.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.9ms
Speed: 2.2ms preprocess, 5.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.7ms
Speed: 2.0ms preprocess, 5.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
Speed: 2.1ms preprocess, 5.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.8ms
Speed: 2.0ms preprocess, 5.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5.7ms
Speed: 2.1ms preprocess, 5.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 5

# Implementing the License Plate Detection

In [5]:
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5,6]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # debugging check if bbox lines up with detected vehicles (should be identical to save_crops() above
                    # cv.imwrite(str(track_id) + '.jpg', roi)
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # verify detections
                        print(license_plate, 'track_id: ' + str(bbox[4]))
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        cv.imwrite(str(track_id) + '.jpg', plate)
                        
video.release()


0: 384x640 1 car, 14.2ms
Speed: 11.8ms preprocess, 14.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
CLASS_ID 2.0
[167.5011444091797, 9.135247230529785, 1225.538330078125, 738.2220458984375, 1.0, 0.9099169373512268]

0: 448x640 1 License_Plate, 64.6ms
Speed: 2.0ms preprocess, 64.6ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
[515.7381591796875, 543.5557861328125, 723.4981079101562, 672.1771240234375, 0.815707266330719, 0.0] track_id: 1.0

0: 384x640 1 car, 5.7ms
Speed: 2.0ms preprocess, 5.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
CLASS_ID 2.0
[41.51203155517578, 4.180763244628906, 1153.928466796875, 757.9590454101562, 1.0, 0.9045620560646057]

0: 448x640 1 License_Plate, 5.5ms
Speed: 2.2ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
[636.5040283203125, 554.1988525390625, 844.3439331054688, 680.4312133789062, 0.8196196556091309, 0.0] track_id: 1.0

0: 384x640 1 car, 5.6ms
Speed

# Preprocess License Plates

Snipping the model

In [15]:
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
vehicles = [2,3,5,6]

# read the 10 first frames
tracker = None 
tracker = {}
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 300:
        
        # vehicle detector
        detections = coco_model.track(frame, persist = True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print("BBOX:", bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        if (not tracker.get(track_id) or tracker.get(track_id) == False) and plate_score > 0.5:
                            cv.imwrite(str(track_id) + '_gray.jpg', plate_gray)
                            cv.imwrite(str(track_id) + '_thresh.jpg', plate_treshold)
                            tracker[track_id] = True  
                        
video.release()
tracker = None 


0: 384x640 2 cars, 16.6ms
Speed: 4.9ms preprocess, 16.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
BBOX: [358.00244140625, 119.390869140625, 1290.63037109375, 713.5140380859375, 38.0, 0.9589166641235352]

0: 416x640 1 License_Plate, 7.8ms
Speed: 2.0ms preprocess, 7.8ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)
BBOX: [1498.9091796875, 112.72341918945312, 1919.0234375, 448.7666931152344, 39.0, 0.8291318416595459]

0: 512x640 (no detections), 7.4ms
Speed: 1.8ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 2 cars, 7.1ms
Speed: 1.8ms preprocess, 7.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
BBOX: [354.5360107421875, 115.42909240722656, 1288.5828857421875, 711.1102905273438, 38.0, 0.9546696543693542]

0: 416x640 1 License_Plate, 7.9ms
Speed: 1.8ms preprocess, 7.9ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)
BBOX: [1497.3846435546875, 106.68123626708

In [ ]:
import easyocr 
reader = easyocr.Reader(['en'], gpu=True)

def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection
        text = text.upper().replace(' ', '')
        return text, score

    return None, None

In [13]:
def write_csv(results, output_path):
    
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

In [17]:
results = {}

# read video by index
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, './results.csv')
video.release()


0: 384x640 1 car, 16.3ms
Speed: 14.7ms preprocess, 16.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[1395.7745361328125, 88.39340209960938, 1920.0, 429.8765869140625, 27.0, 0.8291318416595459]

0: 416x640 (no detections), 10.4ms
Speed: 1.9ms preprocess, 10.4ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

0: 384x640 2 cars, 10.0ms
Speed: 2.4ms preprocess, 10.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
[1475.620361328125, 100.6448974609375, 1920.0, 438.1156005859375, 27.0, 0.8436985015869141]

0: 512x640 (no detections), 10.5ms
Speed: 2.1ms preprocess, 10.5ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
[354.4914245605469, 115.36434936523438, 1288.541259765625, 711.1362915039062, 30.0, 0.9546696543693542]

0: 416x640 1 License_Plate, 10.6ms
Speed: 2.1ms preprocess, 10.6ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 384x640 2 cars, 10.0ms
Speed: 2.1ms preprocess, 10.0ms infer

In [18]:
import pandas as pd 
results = pd.read_csv("results.csv")
display(results)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score
0,1,30.0,[354.4914245605469 115.36434936523438 1288.541...,0.954670,[584.833740234375 237.95245361328125 738.04907...,0.822703,BG74164,0.426392
1,2,30.0,[356.0426330566406 111.84161376953125 1287.490...,0.956794,[580.5357055664062 238.82339477539062 733.6772...,0.827781,BG*74164',0.257307
2,3,30.0,[354.68255615234375 111.03318786621094 1287.11...,0.951902,[580.69140625 236.56112670898438 733.535827636...,0.816828,BG'74164,0.805869
3,4,30.0,[352.4009094238281 108.68882751464844 1286.556...,0.948558,[580.7408447265625 237.8740692138672 734.63098...,0.822905,8G'74164,0.503946
4,5,30.0,[348.1557922363281 108.1434555053711 1284.0415...,0.950611,[582.8560791015625 236.093017578125 735.566589...,0.817071,BG:74164,0.853664
...,...,...,...,...,...,...,...,...
78,94,27.0,[892.203369140625 155.44717407226562 1920.0 66...,0.915013,[799.2359619140625 213.7666473388672 902.95361...,0.740073,5615,0.235703
79,96,27.0,[871.7804565429688 155.13502502441406 1918.571...,0.934325,[140.93150329589844 137.94235229492188 180.464...,0.292818,6C,0.085224
80,97,27.0,[864.7685546875 156.34515380859375 1917.103149...,0.918065,[788.312255859375 213.0118865966797 896.878417...,0.721181,HOOTON,0.032336
81,98,27.0,[857.2617797851562 149.4171905517578 1904.7346...,0.923967,[782.7215576171875 217.1156768798828 891.01898...,0.731399,VP667O,0.105115


In [19]:
for i in results.loc[:, 'license_plate_number']:
    print(i)

BG74164
BG*74164'
BG'74164
8G'74164
BG:74164
IBG:74164
0G'74164
BG'74164!
BG'74164
BG:74164
BG74164
BG*74164
BG74164
BG'74164
BG74164_
BG74164
74164
(BG:74164=
BG741645
BG74164
4164
BG741647
BG'74164'
BG74164
BG'74164
EG'74164
BG'74164
BG74164
8G'74164
BG'74164
BG74164
BG74184
BG'74184
BG74164
BG'74164
8G'74164
IBG'74164'
[BG'74164
86'74164
86*70164
8G'72184
86*74162
8G'74164
BG7416
BG74184
66*74184
BG'70164
0G*74164
BG:70164
BG*74164|
BG74164
8G74164
DG74164
BG*741647
BG*74164
BG74164
EG741641
BG74164
DC*741641
BC741607
741641
80*74164
8C*701641
DC74164'
96741647
2G74160'
BG74164'
172164'
'72164"
4164
184'
54
#U'
#BOTC?
4V66!
HCGIC'
BC|5
061
5615
6C
HOOTON
VP667O
41T6670


# Cleaning Up License Plate Format

In [20]:
import string 
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

def license_complies_format(text):
    # True if the license plate complies with the format, False otherwise.
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

In [21]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

In [22]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        # verify that text is conform to a standard license plate
        if license_complies_format(text):
            # bring text into the default license plate format
            return format_license(text), score

    return None, None

In [29]:
results = {}

# read video by index
video = cv.VideoCapture(videos[1])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the entire video
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if class_id in vehicles and score >= 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }


write_csv(results, './results.csv')
video.release()


0: 384x640 2 cars, 17.4ms
Speed: 12.1ms preprocess, 17.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[475.602294921875, 74.87088012695312, 1358.434814453125, 708.8089599609375, 49.0, 0.9589166641235352]

0: 480x640 1 License_Plate, 12.0ms
Speed: 2.2ms preprocess, 12.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
[1443.09814453125, 101.98013305664062, 1861.380615234375, 445.6099853515625, 61.0, 0.8291318416595459]

0: 544x640 (no detections), 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

0: 384x640 2 cars, 10.6ms
Speed: 1.9ms preprocess, 10.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
[388.28643798828125, 103.7259521484375, 1307.940673828125, 710.2637329101562, 49.0, 0.9546696543693542]

0: 448x640 1 License_Plate, 11.7ms
Speed: 2.2ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
[1483.0391845703125, 104.29417419433594, 19

ValueError: not enough values to unpack (expected 7, got 6)

In [19]:
import pandas as pd 
results = pd.read_csv("/home/jik19004/FilesToRun/5717/results.csv")
display(results)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score
0,24,33.0,[601.1561889648438 4.072883605957031 1750.9892...,0.913115,[662.4776000976562 598.8257446289062 880.41192...,0.814163,BG54JSS,0.666176
1,25,33.0,[653.9697875976562 2.218721866607666 1818.6804...,0.906419,[674.1043701171875 601.7662963867188 893.74353...,0.807751,BG54JSS,0.558057
2,26,33.0,[708.40771484375 2.919558525085449 1887.397827...,0.910795,[683.5406494140625 605.2889404296875 906.42553...,0.807868,BG54JSS,0.789940
3,28,33.0,[812.4795532226562 2.5051324367523193 1920.0 7...,0.930575,[694.0138549804688 604.1604614257812 936.64624...,0.814614,BG54JSS,0.712180
4,268,40.0,[1160.510009765625 178.94122314453125 1918.825...,0.933016,[182.10702514648438 260.2650451660156 291.9093...,0.825137,AK53OAS,0.406800
5,286,40.0,[1043.21435546875 151.48883056640625 1819.2698...,0.918625,[205.92674255371094 279.42138671875 323.863037...,0.818410,AK53OAS,0.263032
6,295,40.0,[921.0422973632812 127.66746520996094 1665.026...,0.907127,[220.57476806640625 295.32586669921875 338.515...,0.826768,AK53OAS,0.186024
7,298,40.0,[864.16455078125 109.02523803710938 1596.64868...,0.937038,[227.0639190673828 296.7500915527344 344.29205...,0.819721,AK53OAS,0.347298
8,299,40.0,[848.61328125 99.23661041259766 1577.499633789...,0.932668,[225.84359741210938 295.74102783203125 346.818...,0.855361,AK53OAS,0.454768
9,302,40.0,[792.7227172851562 79.32235717773438 1515.9315...,0.927014,[232.4665985107422 300.72698974609375 350.7754...,0.827478,AK53OAS,0.246314


In [20]:
import pandas as pd 
results = pd.read_csv('./results.csv')

# show results for tracking ID `1` - sort by OCR prediction confidence
results[results['track_id'] == 1.].sort_values(by='license_text_score', ascending=False)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score


# Ignore Bottom Remaining Code

In [21]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=6, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

# read video by index
video = cv.VideoCapture(videos[1])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'DIVX')
out = cv.VideoWriter('./outputs/processed.avi', fourcc, 20.0, size)

# reset video before you re-run cell below
frame_number = -1
video.set(cv.CAP_PROP_POS_FRAMES, 0)

True

In [22]:
ret = True
frame_number = -1
import ast  
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        df_ = results[results['frame_number'] == frame_number]
        for index in range(len(df_)):
            # draw car
            vhcl_x1, vhcl_y1, vhcl_x2, vhcl_y2 = ast.literal_eval(df_.iloc[index]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            
            draw_border(
                frame, (int(vhcl_x1), int(vhcl_y1)),
                (int(vhcl_x2), int(vhcl_y2)), (0, 255, 0),
                12, line_length_x=200, line_length_y=200)
            
            # draw license plate
            plate_x1, plate_y1, plate_x2, plate_y2 = ast.literal_eval(df_.iloc[index]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

            # region of interest
            roi = frame[int(vhcl_y1):int(vhcl_y2), int(vhcl_x1):int(vhcl_x2)]
            cv.rectangle(roi, (int(plate_x1), int(plate_y1)), (int(plate_x2), int(plate_y2)), (0, 0, 255), 6)

            # write detected number
            (text_width, text_height), _ = cv.getTextSize(
                df_.iloc[index]['license_plate_number'],
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                6)

            cv.putText(
                frame,
                df_.iloc[index]['license_plate_number'],
                (int((vhcl_x2 + vhcl_x1 - text_width)/2), int(vhcl_y1 - text_height)),
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                (0, 255, 0),
                6
            )

        out.write(frame)
        frame = cv.resize(frame, (1280, 720))

out.release()
video.release()